## 1 - Introduction to classic RAG in Azure AI Search

This notebook provides instructions and steps for setting up your development environment, resources, and variables. It also explains how models are used in this series, and how a search index schema is structured for RAG workloads.

Steps in this notebook include:

- Sign in to Azure
- Set up the Azure resources used in the pipeline
- Create a virtual environment
- Install packages
- Set variables for endpoints and models
- Choose and deploy models for vectorization and chat
- Review index schema considerations for classic RAG

When you're finished with these steps, you are ready to set up the indexer pipeline and ingest your content.

Sample data is a collection of PDF pages from the NASA's Earth Book that you load into Azure Storage and retrieve during indexing.

This series assumes embedding and chat models on Azure OpenAI so that you can use the integrated vectorization capabilities of Azure AI Search. You can use a different provider but you might need custom skills or a different approach for indexing and embedding your content.

## Prerequisites

You need the following Azure resources to run all of the script in this notebook.

- [Azure Storage](https://learn.microsoft.com/azure/storage/common/storage-account-create), general purpose account, used for providing the PDFs.

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource) provides the embedding and chat models.

- [Azure AI Services multiservice account](https://learn.microsoft.com/azure/ai-services/multi-service-resource), in the same region as Azure AI Search, used for recognizing location entities in the Earth Book.

- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal), basic tier or higher is recommended. Choose the same region as Azure OpenAI and Azure AI multiservice.

Make sure Azure AI Search, Azure OpenAI, and Azure AI multiservice resources are in the same region. To meet the same-region requirement, start by reviewing the [regions for the embedding and chat models](https://learn.microsoft.com/azure/ai-services/openai/concepts/models#model-summary-table-and-region-availability) you want to use. Once you identify a region, confirm that Azure AI Search with AI services integration is available in the [same region](https://learn.microsoft.com/azure/search/search-region-support#azure-public-regions).

## Sign in to Azure

You might not need this step, but if downstream connections fail with a 401 during indexer pipeline execution, it could be because you're using the wrong tenant or subscription. You can avoid this issue by signing in from the command line, explicitly setting the tenant ID and choosing the right subscription.

This section assumes you have the [Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively).

1. Open a command line prompt.

1. Run this command to get the current tenant and subscription information: `az account show`

1. If you have multiple subscriptions, specify the one that has Azure AI Search and Azure OpenAI: `az account set --subscription <PUT YOUR SUBSCRIPTION ID HERE>`

1. If you have multiple tenants, you can list them: `az account tenant list`

1. Sign in to Azure, specifying the tenant used for Azure AI Search and Azure OpenAI: `az login --tenant <PUT YOUR TENANT ID HERE> `

You should now be logged in to Azure from your local device.

## Set up Azure resources using the Azure portal

We recommend using the Azure portal for setting up resources.

You must be a subscription **Owner** or **User Access Administrator** to create roles. If you don't have permission to create roles, you can use API keys instead. If you're using keys, you can skip the steps that enable system assigned managed identities.

### Configure Azure Storage

1. Download the sample PDF files from [nasa-e-book/earth_book_2019_text_pages](https://github.com/Azure-Samples/azure-search-sample-data/tree/main/nasa-e-book/earth_book_2019_text_pages).

1. Sign in to the [Azure portal](https://portal.azure.com).

1. On the Azure Storage left menu, select **Storage browser** > **Blob containers**, and then **Add container**.

1. Name the container *nasa-ebooks-pdfs-all*.

1. Upload the PDFs to the container.

1. On the left menu, select **Settings** > **Identity** and turn on system assigned managed identity.

### Configure Azure AI Search

1. On the Azure AI Search left menu, select **Settings** > **Semantic ranker** and enable the free plan that authorizes 1,000 requests at no charge.

1. On the left menu, select **Settings** > **Keys** and turn on role-based access control or "both".

1. On the left menu, select **Settings** > **Identity** and turn on system assigned managed identity.

### Configure Azure OpenAI

Deploy the following models on Azure OpenAI:

- text-embedding-3-large on Azure OpenAI for embeddings
- gpt-4o on Azure OpenAI for chat completion

You must have [**Cognitive Services OpenAI Contributor**](https://learn.microsoft.com/azure/ai-foundry/openai/how-to/role-based-access-control?view=foundry-classic#cognitive-services-openai-contributor) or higher to deploy models in Azure OpenAI.

1. Go to [Azure OpenAI Studio](https://oai.azure.com/).

1. Select **Deployments** on the left menu.

1. Select **Deploy model** > **Deploy base model**.

1. Select **text-embedding-3-large** from the dropdown list and confirm the selection.

1. Specify a deployment name. We recommend "text-embedding-3-large".

1. Accept the defaults.

1. Select **Deploy**.

1. Repeat the previous steps for **gpt-4o**.

Make a note of the model names and endpoint. Embedding skills and vectorizers assemble the full endpoint internally, so you only need the resource URI. For example, given `https://MY-FAKE-ACCOUNT.openai.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2024-06-01`, the endpoint you should provide in skill and vectorizer definitions is `https://MY-FAKE-ACCOUNT.openai.azure.com`.

### Configure search engine role-based access to Azure Storage

1. Sign in to the [Azure portal](https://portal.azure.com) and find your storage account.

1. On the left menu, select **Access control (IAM)**.

1. Add a role for **Storage Blob Data Reader**, assigned to the search service system-managed identity.

### Configure search engine role-based access to Azure models

Assign yourself *and* the search service identity permissions on Azure OpenAI. The code for this series runs locally. Requests to Azure OpenAI originate from your system. Also, embedding requests and query responses from the search engine are passed to Azure OpenAI. For these reasons, both you and the search service need permissions on Azure OpenAI.

1. Sign in to the [Azure portal](https://portal.azure.com) and find your Azure OpenAI resource.

1. On the left menu, select **Access control (IAM)**.

1. Add a role for [**Cognitive Services OpenAI User**](https://learn.microsoft.com/azure/ai-foundry/openai/how-to/role-based-access-control?view=foundry-classic#cognitive-services-openai-contributor).

1. Select **Managed identity** and then select **Members**. Find the system-managed identity for your search service in the dropdown list.

1. Next, select **User, group, or service principal** and then select **Members**. Search for your user account and then select it from the dropdown list.

1. Select **Review and Assign** to create the role assignments.

This step concludes provisioning services in the Azure portal. Continuing to the next section, you switch to Visual Studio Code and a local environment.

## Create a virtual environment in Visual Studio Code

Create a virtual environment so that you can install the dependencies in isolation.

1. In Visual Studio Code, open the folder containing 1-introduction-and-setup.ipynb.

1. Press Ctrl-shift-P to open the command palette, search for "Python: Create Environment", and then select `Venv` to create a virtual environment in the current workspace.

1. Select requirements.txt for the dependencies.

It takes several minutes to create the environment. When the environment is ready, continue to the next step.

## Install packages

In [ ]:
! pip install -r requirements.txt --quiet

## Set endpoints

Provide the endpoints you collected in a previous step. You can leave the API keys empty if you enabled role-based authentication. Otherwise, if you can't use roles, provide API keys for each resource.

The Azure AI multiservice account is used for skills processing. The multiservice account key must be provided, even if you're using role-based access control. The key isn't used on the connection, but it's currently used for billing purposes.

In [ ]:
# Set endpoints and API keys for Azure services
AZURE_SEARCH_SERVICE: str = "PUT YOUR SEARCH SERVICE URL HERE"
# AZURE_SEARCH_KEY: str = "DELETE IF USING ROLES, OTHERWISE PUT YOUR SEARCH SERVICE ADMIN KEY HERE"
AZURE_OPENAI_ACCOUNT: str = "PUT YOUR AZURE OPENAI ACCOUNT URL HERE"
# AZURE_OPENAI_KEY: str = "DELETE IF USING ROLES, OTHERWISE PUT YOUR AZURE OPENAI KEY HERE"
AZURE_AI_MULTISERVICE_ACCOUNT: str = "PUT YOUR AZURE AI MULTISERVICE ACCOUNT URL HERE"
AZURE_AI_MULTISERVICE_KEY: str = "PUT YOUR AZURE AI MULTISERVICE KEY HERE. ROLES ARE USED TO CONNECT. KEY IS USED FOR BILLING."
AZURE_STORAGE_CONNECTION: str = "PUT YOUR AZURE STORAGE CONNECTION STRING HERE (see example below for syntax)"

# Example connection string for a search service managed identity connection:
# "ResourceId=/subscriptions/FAKE-SUBCRIPTION=ID/resourceGroups/FAKE-RESOURCE-GROUP/providers/Microsoft.Storage/storageAccounts/FAKE-ACCOUNT;"

## Choose models

A RAG solution built on Azure AI Search takes a dependency on embedding models for vectorization, and on chat completion models for conversational search over your data.

You need a model provider, such as [Azure OpenAI](https://learn.microsoft.com/azure/ai-foundry/openai/how-to/create-resource), Azure Vision in Foundry Tools via a [Microsoft Foundry resource](https://learn.microsoft.com/azure/ai-services/multi-service-resource?pivots=azportal), or the [Foundry model catalog](https://ai.azure.com/?cid=learnDocs). For Azure Vision, ensure that your Foundry resource is in the same region as [Azure AI Search](https://learn.microsoft.com/azure/search/search-region-support) and the [Azure Vision multimodal APIs](https://learn.microsoft.com/azure/ai-services/computer-vision/overview-image-analysis?tabs=4-0#region-availability).

We use Azure OpenAI in this series. Other providers are listed so that you know your options for integrated vectorization.

### Review models supporting built-in vectorization

Vectorized content improves the query results in a RAG solution. Azure AI Search supports a built-in vectorization action in an indexing pipeline. It also supports vectorization at query time, converting text or image inputs into embeddings for a vector search. In this step, identify an embedding model that works for your content and queries. If you're providing raw vector data and raw vector queries, or if your RAG solution doesn't include vector data, skip this step.

Vector queries that include a text-to-vector conversion step must use the same embedding model that was used during indexing. The search engine doesn't throw an error if you use different models, but you get poor results.

To meet the same-model requirement, choose embedding models that can be referenced through *skills* during indexing and through *vectorizers* during query execution. The following table lists the skill and vectorizer pairs. To see how the embedding models are used, skip ahead to [Set up an indexing pipeline](2-build-the-pipeline.ipynb) for code that calls an embedding skill and a matching vectorizer. 

Azure AI Search provides skill and vectorizer support for the following embedding models in the Azure cloud.

| Client | Embedding models | Skill | Vectorizer |
|--------|------------------|-------|------------|
| Azure OpenAI | text-embedding-ada-002<br>text-embedding-3-large<br>text-embedding-3-small | [AzureOpenAIEmbedding](https://learn.microsoft.com/azure/search/cognitive-search-skill-azure-openai-embedding) | [AzureOpenAIEmbedding](https://learn.microsoft.com/azure/search/vector-search-vectorizer-azure-open-ai) |
| Azure Vision | multimodal 4.0 | [AzureAIVision](https://learn.microsoft.com/azure/search/cognitive-search-skill-vision-vectorize) | [AzureAIVision](https://learn.microsoft.com/azure/search/vector-search-vectorizer-ai-services-vision) |
| Foundry model catalog | Cohere-embed-v3-english <br>Cohere-embed-v3-multilingual <br>Cohere-embed-v4 | [AML](https://learn.microsoft.com/azure/search/cognitive-search-aml-skill)  | [Foundry model catalog](https://learn.microsoft.com/azure/search/vector-search-vectorizer-azure-machine-learning-ai-studio-catalog) |

Azure Vision supports text and image vectorization.

At this time, you can only specify `embed-v-4-0` programmatically through the [AML skill](https://learn.microsoft.com/azure/search/cognitive-search-aml-skill) or [Microsoft Foundry model catalog vectorizer](https://learn.microsoft.com/azure/search/vector-search-vectorizer-azure-machine-learning-ai-studio-catalog), not through the Azure portal. However, you can use the portal to manage the skillset or vectorizer afterward.

Deployed models in the model catalog are accessed over an AML endpoint. We use the existing AML skill for this connection.

**NOTE:** Inputs to an embedding models are typically chunked data. In an Azure AI Search RAG pattern, chunking is handled in the indexer pipeline, covered in [another notebook](2-build-the-pipeline.ipynb) in this series.

### Review models used for generative AI at query time

Azure AI Search doesn't have integration code for chat models, so you should choose an LLM that you're familiar with and that meets your requirements. You can modify query code to try different models without having to rebuild an index or rerun any part of the indexing pipeline. Review [Search and generate answers](3-search-and-generate-answers.ipynb) for code that calls the chat model.

The following Azure OpenAI models are commonly used for a chat search experience:

- GPT-4
- GPT-4o
- GPT-4.1
- GPT-5

GPT-4 and GPT-5 models are optimized to work with inputs formatted as a conversation.

We use GPT-4o in this exercise.


## Design an index

An index contains searchable text and vector content, plus configurations. In a classic RAG pattern that uses a chat model for responses, you want an index designed around chunks of content that can be passed to an LLM at query time. This section covers the characteristics of an index schema that works for classic RAG.

In conversational search, LLMs compose the response that the user sees, not the search engine, so you don't need to think about what fields to show in your search results, and whether the representations of individual search documents are coherent to the user. Depending on the question, the LLM might return verbatim content from your index, or more likely, repackage the content for a better answer.

### Organized around chunks

When LLMs generate a response, they operate on chunks of content for message inputs, and while they need to know where the chunk came from for citation purposes, what matters most is the quality of message inputs and its relevance to the user's question. Whether the chunks come from one document or a thousand, the LLM ingests the information or *grounding data*, and formulates the response using instructions provided in a system prompt.

Chunks are the focus of the schema, and each chunk is the defining element of a search document in a RAG pattern. You can think of your index as a large collection of chunks, as opposed to traditional search documents that probably have more structure, such as fields containing uniform content for a name, descriptions, categories, and addresses.

### Enhanced with generated data

In this exercise, sample data consists of PDFs and content from the [NASA Earth Book](https://www.nasa.gov/ebooks/earth/). This content is descriptive and informative, with numerous references to geographies, countries, and areas across the world. All of the textual content is captured in chunks, but recurring instances of place names create an opportunity for adding structure to the index. 

By adding skills, it's possible to recognize entities in the text and capture them in an index for use in queries and filters. We include an [entity recognition skill](https://learn.microsoft.com/azure/search/cognitive-search-skill-entity-recognition-v3) that recognizes and extracts location entities, loading it into a searchable and filterable `locations` field. Adding structured content to your index gives you more options for filtering, improved relevance, and more focused answers.

### Parent-child fields in one or two indexes?

Chunked content typically derives from a larger document. And although the schema is organized around chunks, you also want to capture properties and content at the parent level. Examples of these properties might include the parent file path, title, authors, publication date, or a summary.

An inflection point in schema design is whether to have two indexes for parent and child/chunked content, or a single index that repeats parent elements for each chunk.

In this series, because all of the chunks of text originate from a single parent (NASA Earth Book), you don't need a separate index dedicated to up level the parent fields. However, if you're indexing from multiple parent PDFs, you might want a parent-child index pair to capture level-specific fields and then send [lookup queries](https://learn.microsoft.com/rest//rest/api/searchservice/documents/get) to the parent index to retrieve those fields relevant to each chunk.

### Checklist of schema considerations

In Azure AI Search, an index that works best for RAG workloads has these qualities:

- Returns chunks that are relevant to the query and readable to the LLM. LLMs can handle a certain level of dirty data in chunks, such as mark up, redundancy, and incomplete strings. While chunks need to be readable and relevant to the question, they don't need to be pristine.

- Maintains a parent-child relationship between chunks of a document and the properties of the parent document, such as the file name, file type, title, author, and so forth. To answer a query, chunks could be pulled from anywhere in the index. Association with the parent document providing the chunk is useful for context, citations, and follow up queries.

- Accommodates the queries you want create. You should have fields for vector and hybrid content, and those fields should be attributed to support specific query behaviors, such as searchable or filterable. You can only query one index at a time (no joins) so your fields collection should define all of your searchable content.

- Your schema should either be flat (no complex types or structures), or you should [format the complex type output as JSON](https://learn.microsoft.com/azure/search/search-get-started-rag?pivots=csharp#send-a-complex-rag-query) before sending it to the LLM. This requirement is specific to the RAG pattern in Azure AI Search.

**NOTE:** Schema design affects storage and costs. This exercise is focused on schema fundamentals. In the [Minimize storage and costs](5-minimize-storage-and-costs.ipynb) exercise, you revisit schemas to learn how narrow data types, compression, and storage options significantly reduce the amount of storage used by vectors.

### A minimal index designed for RAG workloads

A minimal index for LLM is designed to store chunks of content. It typically includes vector fields if you want similarity search for highly relevant results. It also includes nonvector fields for human-readable inputs to the LLM for conversational search. Nonvector chunked content in the search results becomes the grounding data sent to the LLM.

Here's a minimal index definition for RAG solutions that support vector and hybrid search. Review it for an introduction to required elements: index name, fields, and a configuration section for vector fields.

```json
{
    "name": "example-minimal-index",
    "fields": [
    { "name": "id", "type": "Edm.String", "key": true },
    { "name": "chunked_content", "type": "Edm.String", "searchable": true, "retrievable": true },
    { "name": "chunked_content_vectorized", "type": "Edm.Single", "dimensions": 1536, "vectorSearchProfile": "my-vector-profile", "searchable": true, "retrievable": false, "stored": false },
    { "name": "metadata", "type": "Edm.String", "retrievable": true, "searchable": true, "filterable": true }
    ],
    "vectorSearch": {
        "algorithms": [
            { "name": "my-algo-config", "kind": "hnsw", "hnswParameters": { }  }
        ],
        "profiles": [ 
        { "name": "my-vector-profile", "algorithm": "my-algo-config" }
        ]
    }
}
```

Fields must include key field (`"id"` in this example) and should include vector chunks for similarity search, and nonvector chunks for inputs to the LLM. 

Vector fields are associated with algorithms that determine the search paths at query time. The index has a vectorSearch section for specifying multiple algorithm configurations. Vector fields also have [specific types](https://learn.microsoft.com/rest/api/searchservice/supported-data-types#edm-data-types-for-vector-fields) and extra attributes for embedding model dimensions. `Edm.Single` is a data type that works for commonly used LLMs. For more information about vector fields, see [Create a vector index](https://learn.microsoft.com/azure/search/vector-search-how-to-create-index?tabs=push%2Cportal-check-index).

Metadata fields might be the parent file path, creation date, or content type and are useful for [filters](https://learn.microsoft.com/azure/search/vector-search-filters?tabs=prefilter-mode).

### The index schema for this series

Here's the index schema for the [Earth Book content](https://github.com/Azure-Samples/azure-search-sample-data/tree/main/nasa-e-book/earth_book_2019_text_pages) used in this series.

Like the basic schema, it's organized around chunks. The `chunk_id` uniquely identifies each chunk. The `text_vector` field is an embedding of the chunk. The nonvector `chunk` field is a readable string. The `title` maps to a unique metadata storage path for the blobs. The `parent_id` is the only parent-level field, and it's a base64-encoded version of the parent file URI. 

In integrated vectorization workloads like the one used in this series, the `dimensions` property on your vector fields should be identical to the number of `dimensions` generated by the embedding skill used to vectorize your data. In this series, we use the Azure OpenAI embedding skill, which calls the text-embedding-3-large model on Azure OpenAI. The skill is specified in the next exercise. We set dimensions to 1024 in both the vector field and in the skill definition.

The schema also includes a `locations` field for storing generated content that's created by the [indexing pipeline](2-build-the-pipeline.ipynb).

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SearchIndex
)

credential = DefaultAzureCredential()

# Create a search index  
index_name = "py-rag-tutorial-idx"
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE, credential=credential)  
fields = [
    SearchField(name="parent_id", type=SearchFieldDataType.String),  
    SearchField(name="title", type=SearchFieldDataType.String),
    SearchField(name="locations", type=SearchFieldDataType.Collection(SearchFieldDataType.String), filterable=True),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True, analyzer_name="keyword"),  
    SearchField(name="chunk", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),  
    SearchField(name="text_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1024, vector_search_profile_name="myHnswProfile")
    ]  
    
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
            vectorizer_name="myOpenAI",  
        )
    ],  
    vectorizers=[  
        AzureOpenAIVectorizer(  
            vectorizer_name="myOpenAI",  
            kind="azureOpenAI",  
            parameters=AzureOpenAIVectorizerParameters(  
                resource_url=AZURE_OPENAI_ACCOUNT,  
                deployment_name="text-embedding-3-large",
                model_name="text-embedding-3-large"
            ),
        ),  
    ], 
)  
    
# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")  